## 1.可视化卷积神经网络的中间输出（中间激活）：

载入模型信息

In [ ]:

from keras.models import load_model
# import tensorflow as tf
# 注意这个地址
model = load_model("/home/wufisher/my_tensorflow_learn/python/tensorflow/day10/candd_v2.h5")
print(model.summary())

输入一张😺的测试图像

In [ ]:
img_path = '/home/wufisher/dataset_m/mm_and_dd/cats_and_dogs_small/test/cats/1700.jpg'
from keras.preprocessing import image
import numpy as np
img = image.load_img(img_path,target_size=(150,150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor,axis=0)
img_tensor/=255.
print(img_tensor.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img_tensor[0])
plt.show()

为了提取想要查看的特征图，我们需要创建一个Keras模型，以图像批量作为输入，输出所有卷积层和池化层的激活
model类允许多个输出，与Sequential模型不大一样

In [ ]:
from keras import models

layer_outputs = [layer.output for layer in model.layers[:8]]
activation_model=models.Model(inputs=model.input,outputs=layer_outputs)
activations= activation_model.predict(img_tensor)
first_layer_activation = activations[0]
print(first_layer_activation.shape)

将第4个通道可视化(第一层激活的四个通道)

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(first_layer_activation[0,:,:,4],cmap='viridis')

在看一下第七个，这个是不确定的因为每一个层学到的过滤器不一样

In [ ]:
plt.matshow(first_layer_activation[0,:,:,7],cmap='viridis')

将每个通道都可视化

In [ ]:
layer_names=[]
for layer in model.layers[:8]:
    layer_names.append(layer.name)
images_per_row=16

for layer_name, layer_activation in zip(layer_names,activations):
    n_features = layer_activation.shape[-1]

    size= layer_activation.shape[1]

    n_cols= n_features//images_per_row
    display_grid = np.zeros((n_cols*size,images_per_row*size))
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image= layer_activation[0,:,:,col*images_per_row+row]

            channel_image-=channel_image.mean()
            channel_image/=channel_image.std()
            channel_image*=64
            channel_image+=128
            channel_image=np.clip(channel_image,0,255).astype('uint8')
            display_grid[col*size:(col+1)*size,row*size:(row+1)*size]=channel_image

        scale=1./size
        plt.figure(figsize=(scale*display_grid.shape[1],scale*display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid,aspect='auto',cmap='viridis')



在前面几层，激活几乎保存了所有的信息
随着层数加深，激活越来越抽象，视觉内容越来越少，类别信息越来越多
激活的稀疏度随着层数加深而增大，越来越多过滤器是空白的，输入图像中找不到这些过滤器的编码格式